# Compiling CUDA with OpenCV
In this notebook we will show how to compile CUDA C++ code with the OpenCV library, which we will use to load an image and save it back to disk after applying a blur kernel.


## Setup

Install and load the nvcc4jupyter extension, which will allow us to run CUDA C++ code using cell magics. See [the documentation](https://nvcc4jupyter.readthedocs.io/en/latest/usage.html) for details.

In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Make sure OpenCV is installed.

In [ ]:
%%capture
!apt update && apt install -y libopencv-dev

## Code

### Imports

In [ ]:
import os
import subprocess
from pathlib import Path
from IPython.display import Image

### Download an image

Get a random image of IMG_SIZE x IMG_SIZE pixels.

In [ ]:
IMG_SIZE = 400
os.environ["IMG_SIZE"] = str(IMG_SIZE)
IMG_FNAME = "image.jpg"
os.environ["IMG_FNAME"] = IMG_FNAME

!wget -O $IMG_FNAME https://picsum.photos/$IMG_SIZE.jpg &> /dev/null
Image(filename=IMG_FNAME)

### Compiler arguments
These are options that you need to pass to the compiler in order to link with the OpenCV library.

In [ ]:
# make sure the file containing the necessary compilation flags for the opencv library
# exists in a directory in the PKG_CONFIG_PATH environment variable
for path in Path('/usr/lib').rglob('opencv4.pc'):
    os.environ["PKG_CONFIG_PATH"] = os.path.dirname(str(path))
    break
    
# get the compilation flags required to compile our CUDA C++ code with opencv
COMPILER_ARGS = subprocess.check_output(["pkg-config", "--cflags", "--libs", "opencv4"]).decode().strip()
COMPILER_ARGS = f"-I/usr/include/opencv4 {COMPILER_ARGS}"
print(COMPILER_ARGS)

### Blurring the image

In [ ]:
%%cuda --compiler-args "$COMPILER_ARGS"

#include <opencv2/core.hpp>
#include <opencv2/imgcodecs.hpp>

#include <iostream>

#define BLUR_SIZE 5
#define R 0
#define G 1
#define B 2

// error checking macro
#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)

// kernel taken from https://stackoverflow.com/a/65973288
__global__ void blurKernel(
    unsigned char* in,
    unsigned char* out,
    int width,
    int height,
    int num_channel,
    int channel
) {

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if(col < width && row < height) {
        int pixVal = 0;
        int pixels = 0;
        for(int blurRow = -BLUR_SIZE; blurRow < BLUR_SIZE + 1; ++blurRow) {
            for(int blurCol = -BLUR_SIZE; blurCol < BLUR_SIZE + 1; ++blurCol) {
                int curRow = row + blurRow;
                int curCol = col + blurCol;
                if(curRow > -1 && curRow < height && curCol > -1 && curCol < width) {
                    pixVal += in[curRow * width * num_channel + curCol * num_channel + channel];
                    pixels++;
                }
            }
        }
        out[row * width * num_channel + col * num_channel + channel] = (unsigned char)(pixVal/pixels);
    }
}

int main()
{
    std::string image_path = cv::samples::findFile("image.jpg");
    cv::Mat img = imread(image_path, cv::IMREAD_COLOR);
    if(img.empty())
    {
        std::cerr << "Could not read the image: " << image_path << std::endl;
        return 1;
    }

    // image shape
    int width = img.cols;
    int height = img.rows;
    int n_channels = 3; // hard-coded RGB processing

    // we will read and write directly into the data array of the OpenCV Matrix
    unsigned char *host_image_input = img.data;
    unsigned char *host_image_output = img.data;

    // allocate memory for device arrays
    unsigned char* dev_image_input = NULL;
    unsigned char* dev_image_output = NULL;
    cudaMalloc(
        (void**)&dev_image_input,
        sizeof(unsigned char) * height * width * n_channels
    );
    cudaMalloc(
        (void**)&dev_image_output,
        sizeof(unsigned char) * height * width * n_channels
    );
    cudaCheckErrors("cudaMalloc failure");

    // transfer data from host to device for processing on GPU
    cudaMemcpy(
        dev_image_input,
        host_image_input,
        sizeof(unsigned char) * height * width * n_channels,
        cudaMemcpyHostToDevice
    );
    cudaCheckErrors("cudaMemcpy H2D failure");

    // run a blur kernel on each channel
    dim3 blockSize(16, 16, 1);
    dim3 gridSize(width/blockSize.x, height/blockSize.y, 1);
    blurKernel<<<gridSize, blockSize>>>(dev_image_input, dev_image_output, width, height, n_channels, R);
    blurKernel<<<gridSize, blockSize>>>(dev_image_input, dev_image_output, width, height, n_channels, G);
    blurKernel<<<gridSize, blockSize>>>(dev_image_input, dev_image_output, width, height, n_channels, B);

    cudaDeviceSynchronize();
    cudaCheckErrors("kernel failure");

    // copy results back to host
    cudaMemcpy(
        host_image_output,
        dev_image_output,
        sizeof(unsigned char) * height * width * n_channels,
        cudaMemcpyDeviceToHost
    );
    cudaCheckErrors("cudaMemcpy D2H failure");

    cudaFree(dev_image_input);
    cudaFree(dev_image_output);

    // save the blurred image to disk
    cv::imwrite("image_blurred.jpg", img);
    return 0;
}

In [ ]:
Image(filename="image_blurred.jpg")